In [1]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-04-26 06:04:17.288378: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-26 06:04:17.501021: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

In [2]:
from datasets import load_from_disk

source_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
target_data_eval=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data_eval")
test_target_data=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target=load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


In [3]:
from torch.utils.data import DataLoader
source_data_loader = DataLoader(source_data, batch_size=32, shuffle=True)
target_data_loader = DataLoader(target_data, batch_size=32)
source_data_eval_loader = DataLoader(source_data_eval, batch_size=32, shuffle=True)
target_data_eval_loader = DataLoader(target_data_eval, batch_size=32)

target_test_loader = DataLoader(test_target_data, batch_size=32)

In [4]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [5]:
import adapters.composition as ac

domain_adapter_coral = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/domain_adapter_telephone_travel",with_head=False)
mlm_adapter = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/mlm-after-coral",with_head=False)
task_adapter = model.load_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/stacked-adapter-task-modefied",with_head=False)


In [6]:
model.add_adapter_fusion(ac.Fuse(mlm_adapter, task_adapter))
model.add_classification_head("mnli", num_labels=3)
adapter_setup_fuse = ac.Fuse( mlm_adapter, task_adapter)
model.train_adapter_fusion(adapter_setup_fuse)

#model.active_adapters = ac.Stack(domain_adapter_coral, ac.Fuse(mlm_adapter, task_adapter))
model.active_adapters = ac.Fuse(mlm_adapter, task_adapter)



In [7]:
fn.print_trainable_parameters(model)

trainable params: 11841597 || all params: 82644861 || trainable%: 14.32829199144034


In [8]:
model.adapter_summary(as_dict=True)

[{'name': 'domain_adapter_telephone-travel',
  'architecture': 'bottleneck',
  'active': False,
  '#param': 447264,
  'train': False,
  '%param': 0.5809458731520459},
 {'name': 'mlm-after-coral',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 3545856,
  'train': False,
  '%param': 4.605670051672885},
 {'name': 'stacked-adapter-task-modefied',
  'architecture': 'bottleneck',
  'active': True,
  '#param': 447264,
  'train': False,
  '%param': 0.5809458731520459},
 {'name': 'Full model', '#param': 76988928, '%param': 100.0, 'train': False}]

In [9]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
print(model.active_adapters)

distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.query.weight
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.query.bias
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.key.weight
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.key.bias
distilbert.transformer.layer.0.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.value.weight
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.query.weight
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.query.bias
distilbert.transformer.layer.1.output_adapters.adapter_fusion_layer.mlm-after-coral,stacked-adapter-task-modefied.key.wei

In [10]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.37955465587044535
F1 score before adaptation: 0.379429807529451


In [11]:

reload(fn)
trainer = fn.train_model(model,"multi_block",source_data,source_data_eval)

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/9380 [00:00<?, ?it/s]

{'loss': 0.7055, 'learning_rate': 0.0001, 'epoch': 0.21}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.8023195266723633, 'eval_accuracy': 0.6519715578539108, 'eval_f1': 0.6505233334371221, 'eval_precision': 0.7202749221307585, 'eval_recall': 0.6519715578539108, 'eval_runtime': 15.5309, 'eval_samples_per_second': 498.04, 'eval_steps_per_second': 15.582, 'epoch': 0.21}
{'loss': 0.62, 'learning_rate': 9.436936936936938e-05, 'epoch': 0.43}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.6790050268173218, 'eval_accuracy': 0.7100193923723336, 'eval_f1': 0.7109180774214868, 'eval_precision': 0.724771451692845, 'eval_recall': 0.7100193923723336, 'eval_runtime': 15.8187, 'eval_samples_per_second': 488.978, 'eval_steps_per_second': 15.298, 'epoch': 0.43}
{'loss': 0.608, 'learning_rate': 8.873873873873875e-05, 'epoch': 0.64}


  0%|          | 0/242 [00:00<?, ?it/s]

{'eval_loss': 0.6839444637298584, 'eval_accuracy': 0.7142857142857143, 'eval_f1': 0.7141066378849438, 'eval_precision': 0.724784604402491, 'eval_recall': 0.7142857142857143, 'eval_runtime': 16.0477, 'eval_samples_per_second': 481.999, 'eval_steps_per_second': 15.08, 'epoch': 0.64}


KeyboardInterrupt: 

In [17]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7160931174089069
F1 score before adaptation: 0.7176271232227095


In [40]:
# Evaluate the model on the target domain before adaptation
# this was with coral, it means before last edit. see the commented line above
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(trainer.model, target_test_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7130566801619433
F1 score before adaptation: 0.7149068389081805
